##1. Acessando o arquivo _csv_ que contém a composição das receitas da base de dados `CulinaryDB`

In [1]:
import pandas as pd
import re

url = 'https://raw.githubusercontent.com/lamevv/projeto/main/project-2-final/data/raw/culinarydb/04_Recipe-Ingredients_Aliases.csv'
df = pd.read_csv(url)

df.head()

,Recipe ID,Original Ingredient Name,Aliased Ingredient Name,Entity ID
0,1,capsicum,capsicum,362
1,1,green bell pepper,pepper bell,362
2,1,soy sauce,soy sauce,291
3,1,sunflower oil,sunflower,426
4,2,buttermilk,buttermilk,61


##2. Extração inicial das unidades de medidas utilizadas para representar a participação dos ingredientes nas receitas

In [2]:
def extrai_quantidade_e_unidade(row):
    padrao = r'(\d+(?:\.\d+)?\s?\(?\d*[/]?\d*\)?)(?:\s+)?(\w+)'
    texto = str(row['Original Ingredient Name'])
    buscaPadrao = re.search(padrao, texto)

    if buscaPadrao:
        quantidade = buscaPadrao.group(1)
        unidade = buscaPadrao.group(2)
        return quantidade.strip(), unidade.strip()
    else:
        return "NaN", "NaN"

# Aplicando a função 'extrai_quantidade_e_unidade' para os valores da coluna 'Original Ingredient Name'
df[['Quantity', 'Unit']] = df.apply(extrai_quantidade_e_unidade, axis=1, result_type = 'expand')

##3. Análise das unidades de medida identificadas

In [3]:
# Dicionário vazio para armazenar as unidades e seus índices
unidades_indices = {}

# Itera sobre o DataFrame para criar o dicionário
for indice, linha in df.iterrows():
    unidade = linha['Unit']
    if unidade not in unidades_indices:
        unidades_indices[unidade] = [indice]
    else:
        unidades_indices[unidade].append(indice)

unidades_ord = dict(sorted(unidades_indices.items()))

for indice in unidades_ord.keys():
    print(indice, end = ", ")
print()

0, 1, 10, 12, 14, 15, 16, 17, 2, 222, 247, 28, 3, 4, 5, 6, 7, 8, 9, A04, Ahi, Alaska, Alaskan, All, Almond, American, Anaheim, Angel, Anjou, Asian, Atlantic, Australian, Avocado, Avocados, BAY, Bahaman, Banana, Bartlett, Basil, Belgian, Bermuda, Best, Bibb, Biscuit, Black, Boneless, Bosc, Boston, Box, Braeburn, Brown, Brussels, Buffalo, Butter, Buttermilk, C, CLAUSSEN, Calabrese, Calabrian, California, Callisons, Camembert, Camomile, Candied, Carr, Cavaillon, Celestial, Cemitas, Ceylon, Champagne, Chee, Chilean, Chinese, Chocolate, Classic, Clementine, Coconut, Colorado, Comice, Confit, Cornish, Cox, Crenshaw, Creole, Crispy, Cuban, Cubanelle, Cubano, Deep, Delicious, Dietz, Dungeness, Earl, Eastern, Eggs, Empanada, English, European, Extra, FUJI, Flaky, Florida, Flying, Forelle, Forelli, French, Fresno, Fuji, Fuyu, Gaeta, Gala, Galia, Garlic, Genovese, Glass, Globe, Golden, Good, Goya, Granny, Greek, Green, HONEY, Haas, Habenero, Hakurei, Halibut, Hard, Hass, Hatch, Hawaiian, Hillshir

Após a busca inicial, identificamos várias unidades de medida, incluindo algumas inválidas, como valores numéricos, substantivos e adjetivos relacionados a ingredientes, bem como diferentes denominações para uma mesma unidade (por exemplo, tablespoon e TBS). Isso revela que os dados contidos na coluna `Original Ingredient Name` não seguem uma padronização específica, o que dificulta o processamento dessas informações.

Aplicando funções de correção paras os casos que apresentaram "unidades de medidas" numéricas (inválidas):

In [4]:
for X in [12, 14, 15, 16, 17]: # Correção para o padrão 'X 1/2 ounce ...' não identificado pelo regex, com X no conjunto [12, 14, 15, 16, 17]
    for index in unidades_indices[str(X)]:
        df.loc[index, ['Quantity', 'Unit']] = [round(X / 2.0, 2), "ounce"]
    unidades_indices.pop(str(X)) # Remove o valor numérico que foi ajustado do dicinário de unidades

In [5]:
def padrao_correcao_A(texto): # Correção para o padrão 'X (Y-<unite_name>)'
    padrao_A = r'^(\d+)\s*\((\d+(?:\.\d+)?)\s*([a-zA-Z\-]+)[^\w\s]*\)'
    matches = re.search(padrao_A, texto)

    if matches:
        multiplicador = float(matches.group(1))
        quantidade = float(matches.group(2))
        if matches.group(3)[0] == '-':
            unidade = matches.group(3)[1::]
        else:
            unidade = matches.group(3).strip()
        return round(multiplicador * quantidade, 2), unidade
    else:
        return "NaN", "NaN"

def padrao_correcao_B(texto): # Correção para o padrão '<fraction> (Y-<unite_name>)'
    padrao_B = r'^([\d\s]+(?:\/\d+)?(?:\s\d+(?:\/\d+)?)?)\s+(\d+(?:\.\d+)?)\s*([a-zA-Z\-]+)'
    matches = re.search(padrao_B, texto)

    if matches:
        multi = matches.group(1).split()
        multiplicador = eval(multi[0])
        if len(multi) > 1:
            multiplicador += eval(multi[1])
        quantidade = float(matches.group(2))
        unidade = matches.group(3)
        return round(multiplicador * quantidade, 2), unidade[1::].strip()
    else:
        return "NaN", "NaN"

for unidade, indices in unidades_indices.items():
    if unidade.isnumeric():
        for indice in indices:  # Corrigido para iterar sobre os índices
            q_A, u_A = padrao_correcao_A(str(df.loc[indice, 'Original Ingredient Name']))
            if q_A == 'NaN' and u_A == 'NaN':
                q_B, u_B = padrao_correcao_B(str(df.loc[indice, 'Original Ingredient Name']))
                df.loc[indice, ['Quantity', 'Unit']] = [q_B, u_B]
            else:
                df.loc[indice, ['Quantity', 'Unit']] = [q_A, u_A]

## 4. Normalização das unidades de medida da Base de Dados

A variável `unidades_de_medida` apresenta o conjunto final das unidades selecionadas para representar a participação de cada ingrediente. Por outro lado, o dicionário `subs` contém abreviações, erros de digitação ou sinônimos das unidades que serão substituídos pelas unidades _padrão_ de medida  associadas. No caso em que a unidade apresentada não esteja contemplada em nenhuma dessas coleções, ou seja, quando a unidade associada a um ingrediente se refere a um substantivo ou adjetivo específico desse item, ela será substituída por `unit`, indicando uma unidade genérica para esse produto.

In [6]:
unidades_de_medida = {'NaN', 'teaspoon', 'tablespoon', 'pound', 'ounce', 'cup', 'bottle', 'can', 'container', 'gram', 'kilogram', 'liter', 'milliliter', 'pinch', 'pint', 'quart', 'scoop'}

subs = {
    'teaspoon': {'teaaspoon', 'teapoon', 'teapsoon', 'teaspon', 'teaspoon', 'teaspoons', 'teaspooon', 'tespoon', 'tsp', 'tsps', 'American'},
    'tablespoon': {'tablesoon', 'tablesoons', 'tablespoona', 'tablespooncandied', 'tablespoonfuls', 'tablespoons', 'tablespoonsClarified', 'tablespoonsgranulated',
          'tablespoonsunsalted', 'tablespoonsunsweetened', 'tablespooons', 'tbs', 'tbsp', 'tbsps', 'TBS', 'TBSP', 'Tablespoon', 'Tablespoon', 'Tablespoons', 'Tbs', 'Tbsp'},
    'pound': {'Pounds', 'lb', 'lbs', 'pounds', 'pound120', 'pouns'},
    'ounce': {'Ounce', 'ounces', 'ouncres', 'oz'},
    'cup': {'C', 'c', 'cups', 'cupsFresh', 'cupsMexican', 'cupsRich', 'cupsWinter', 'cupsall', 'cupsespagnole', 'cupslow', 'cupstomato', 'cupunsweetened', 's'},
    'bottle': {'bottlecapful'},
    'can': {'cans'},
    'container': {'containers'},
    'gram': {'g', 'gm', 'grams'},
    'kilogram': {'kg', 'kilograms'},
    'liter': {'L', 'liters', 'litre', 'litres'},
    'milliliter': {'mL', 'ml', 'milliliters'},
    'bottle': {'bottlecapful', 'bottles'},
    'pinch': {'pinches'},
    'pint': {'pints'},
    'quart': {'qts', 'quartered', 'quarters', 'quarter', 'quarts', 'quartt'},
    'scoop': {'scoops'}
}

for indice, linha in df.iterrows():
    unidade = linha['Unit']
    if unidade not in unidades_de_medida:
        foiSubstituida = False
        for u_padrao, sub in subs.items():
            if unidade in sub:
                df.loc[indice, 'Unit'] = u_padrao
                foiSubstituida = True
                break
        if not foiSubstituida:
            df.loc[indice, 'Unit'] = 'unit'

## 5. Normalização dos valores fracionários da coluna `Quantity` e correção das inconsistências de valores com caracteres não numéricos

In [ ]:
def normaliza_quantity(valor):
    valor = str(valor)
    if '(' in valor:
        if valor[-1] == '(':
            return valor[::-1].strip()
        else:
            partes = valor.split('(')
            return partes[-1].strip()
    elif '/' in valor:
        if valor[-1] == '/':
            return valor[::-1].strip()
        elif ' ' in valor:
            partes = valor.split(' ')
            try:
                num = float(partes[0]) + eval(partes[1])
            except:
                try:
                    return round(eval(partes[0] + partes[1]), 2)
                except:
                    return valor
            return round(num, 2)
        else:
            fracao = valor.split('/')
            try:
                num = int(fracao[0]) / int(fracao[1])
            except:
                return valor
            return round(num, 2)
    else:
        return valor

df['Quantity'] = df['Quantity'].apply(normaliza_quantity)

df.to_csv("Recipe-Ingredients-With-Quantity.csv", index = False)

## 6. Padronização do valor em gramas das unidades de medida selecionadas

Para efeitos de simplificação, utilizaremos uma **tabela de conversão padrão**. Cada uma das unidades listadas em `unidades_de_medida` terá sua equivalência em gramas. Isso unifica e facilita a verificação da composição e participação dos ingredientes nas receitas. Abaixo, encontra-se a tabela com as conversões adotadas:

<center>

| **Unit of Measurement** | **Approximation in grams** |
|:-----------------------:|:--------------------------:|
|         teaspoon        |              5             |
|        tablespoon       |             15             |
|          pound          |             454            |
|          ounce          |             28             |
|           cup           |             250            |
|          bottle         |             750            |
|           can           |             355            |
|        container        |             100            |
|           gram          |              1             |
|         kilogram        |            1000            |
|          liter          |            1000            |
|        milliliter       |              1             |
|          pinch          |            0.35            |
|           pint          |             473            |
|          quart          |             946            |
|          scoop          |             24             |
|           unit          |            100             |

</center>

Algumas unidades, como bottle, can e container, podem variar significativamente dependendo do que contém. Utilizamos a água como elemento principal para as estimativas em gramas. Para a unidade genérica `unit` estipulamos um valor arbitrário de *100* gramas.